In [ ]:
import copy
import os
import pickle
import cv2

import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import math

from mmdet3d.core.utils import visualize_camera, visualize_lidar
from mmdet3d.core.bbox import LiDARInstance3DBoxes

from typing import List, Optional, Tuple
import point_cloud_utils as pcu

from tools.data_converter.lidar_converter import LidarConverter

def display_range(range):
    plt.figure(figsize=(20, 10))
    plt.imshow(range)
    plt.show()


### Range view

In [ ]:
lidar_scan = np.load("./data/nuscenes/nuscenes_pbe_gt_database_train/sample-ca9a282c9e77460f8360f564131a8af5_lidar.npy")
points = lidar_scan[:, :3]

lidar_top_viz = visualize_lidar(points, xlim=(-30, 30), ylim=(-30, 30))
plt.figure(figsize=(20, 20))
plt.imshow(lidar_top_viz)
plt.axis('off')
plt.show()

points.shape

lidar_converter = LidarConverter(W=1096, H=32)

In [ ]:
print("Before resize")
# mask - 1 if point is in the depth interval, 0 otherwise
proj_range, proj_feature, mask, range_pitch, range_yaw = lidar_converter.pcd2range(points, lidar_scan[:, 3])
display_range(proj_range)
display_range(range_pitch)
display_range(range_yaw)

# print("After resize")
# proj_range_resized, proj_feature_resized, _, _ = lidar_converter.resize(proj_range, proj_feature, new_H=64)
# display_range(proj_range_resized)

# print("Reconstructed")
# proj_range_resized, proj_feature_resized, _, _ = lidar_converter.resize(proj_range_resized, proj_feature_resized)
# display_range(proj_range_resized)

lidar_recon, _ = lidar_converter.range2pcd(proj_range, range_pitch, range_yaw)
proj_range, proj_feature, _, range_pitch, range_yaw  = lidar_converter.pcd2range(lidar_recon)
display_range(proj_range)

lidar_recon, _ = lidar_converter.range2pcd(proj_range, range_pitch, range_yaw)
proj_range, proj_feature, _, range_pitch, range_yaw = lidar_converter.pcd2range(lidar_recon)
display_range(proj_range)

In [ ]:
dists, corrs = pcu.k_nearest_neighbors(points[mask], lidar_recon, 1)

In [ ]:
perm = [0, 2, 1]
lidar_top_viz = visualize_lidar(points[mask][dists > 0.001][:, perm], xlim=(-20, 20), ylim=(-20, 20), dpi=30, points_color=(255, 0, 0))
lidar_top_viz_recon = visualize_lidar(lidar_recon[:, perm], xlim=(-20, 20), ylim=(-20, 20), dpi=30)

# Plot them side by side
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.title(f"Original | points {len(points[mask])}")
plt.imshow(lidar_top_viz)
plt.axis('off')
plt.subplot(1, 2, 2)
plt.title(f"Reconstructed | points {len(lidar_recon)}")
plt.imshow(lidar_top_viz_recon)
plt.axis('off')

# overlay the two images
plt.figure(figsize=(20, 20))
plt.imshow(lidar_top_viz)
plt.imshow(lidar_top_viz_recon, alpha=0.5)
plt.axis('off')


plt.show()

print("Percentage of points dropped", np.round(1 - len(lidar_recon) / len(points[mask]), 4) * 100)


print(pcu.chamfer_distance(lidar_recon, points[mask]), lidar_recon.shape, points[mask].shape)

In [ ]:
len(dists < 0.001)

### Benchmark reconstruction - old

In [ ]:
import glob
lidar_scans = glob.glob("/mnt/data/mobi/mit-bevfusion/data/nuscenes/nuscenes_pbe_gt_database_val/*lidar.npy")

ws = [1088, 1090, 1096] #2 ** np.arange(6, 13)
results = {h: {w : [] for w in ws} for h in [32, 128]} 

for i, lidar_scan in enumerate(lidar_scans):
    if i % 10 == 0:
        print(f"Processing {i}/{len(lidar_scans)}")
    points = np.load(lidar_scan)[:, :3].astype(np.float32)

    for w in ws:
        for h in results.keys():
            lidar_converter = LidarConverter(W=w)
            proj_range, proj_feature, mask, range_pitch, range_yaw = lidar_converter.pcd2range(points)
            if h != 32:
                proj_range, proj_feature, _, _ = lidar_converter.resize(proj_range, proj_feature, new_W=w, new_H=h)
                proj_range, proj_feature, _, _ = lidar_converter.resize(proj_range, proj_feature, new_W=w, new_H=32)

            lidar_recon, _ = lidar_converter.range2pcd(proj_range, range_pitch, range_yaw)

            chamfer_dist = pcu.chamfer_distance(lidar_recon, points[mask])
            rel_density = lidar_recon.shape[0] / points[mask].shape[0]
            dropped_points = np.round((1 - rel_density) * 100, 2)
            results[h][w].append(dropped_points)


In [ ]:
plt.figure(figsize=(10, 7))
plt.title("Dropped points (%) in reconstructed point cloud")
for h, w in results.items():
    mean = [np.mean(v) for v in w.values()]
    std = [np.std(v) for v in w.values()]
    plt.errorbar(list(w.keys()), mean, yerr=std, label=h)
plt.xlabel("Width of the range view (log2)")
plt.ylabel("Dropped points (%)")
plt.xscale('log', base=2)
plt.legend()
plt.show()


In [ ]:
np.mean(results[32][1096])

In [ ]:
range_depth = np.load("/mnt/data/mobi/mit-bevfusion/data/nuscenes/nuscenes_pbe_gt_database_train/sample-0cd661df01aa40c3bb3a773ba86f753a_range_depth.npy")
range_insensity = np.load("/mnt/data/mobi/mit-bevfusion/data/nuscenes/nuscenes_pbe_gt_database_train/sample-0cd661df01aa40c3bb3a773ba86f753a_range_intensity.npy")

In [ ]:
print(range_insensity.min(), range_insensity.max())
print(range_depth.min(), range_depth.max())
range_depth.shape

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(range_depth, cmap="turbo")
plt.show()

plt.figure(figsize=(20, 10))
plt.imshow(range_insensity, cmap="turbo")
plt.show()